# 상위 100명의 선호도 정보 저장

In [1]:
import pandas as pd

def _df_load(path, state='train'):
    df = pd.read_csv(path)
    df_count = df.groupby(['응답자 ID']).size()
    df_count.name = f'{state} 설문 응답 수'

    return df, df_count

def _top100_filtering(df, top100_ids, state='train', save=True):
    top100_df = df[df['응답자 ID'].isin(top100_ids)].reset_index(drop=True)
    if save:
        top100_df.to_csv(f'top100_{state}_preference.csv', index=False)
    
    return top100_df

def make_top100_csv(train_csv, val_csv, save=True):
    # csv 파일로부터 데이터 불러오기
    df_train, train_count = _df_load(train_csv, 'train')
    df_val, val_count = _df_load(val_csv, 'val')

    # 몇 가지 전처리
    df_sum = pd.concat([train_count, val_count],axis=1)
    df_sum = df_sum.fillna(0).astype(int)    # 결측치 0으로 채우기
    df_sum['합계'] = df_sum['train 설문 응답 수'] + df_sum['val 설문 응답 수']    # '합계' 열 추가
    df_sum = df_sum.sort_values(by='합계', ascending=False)    # '합계' 열 기준으로 내림차순 정렬

    # df_sum의 합계를 기준으로 상위 100개 응답자 ID 추출하여 리스트로 저장
    top100_ids = df_sum.head(100).index.tolist()

    # 상위 100개의 유효한 응답자 ID를 가진 데이터만 추출
    top100_train_df = _top100_filtering(df_train, top100_ids, 'train', save=save)
    top100_val_df = _top100_filtering(df_val, top100_ids, 'val', save=save)

    return top100_train_df, top100_val_df


# Mission 2-2에서 생성한 csv 파일의 경로
t_pref = 'train_preference.csv'
v_pref = 'val_preference.csv'

#t_top100_pref, v_top100_pref = make_top100_csv(t_pref, v_pref, save=True)

In [2]:
# 결과 보기 좋게 HTML편집
from IPython.display import display_html
def display_left(*args):
    html_str = ''
    for df in args:
        html_str += f'<div style="margin-right:30px;">{df.to_html()}</div>'
    display_html(f'<div style="display: flex;">{html_str}</div>', raw=True)

# 데이터 결과 미리 확인
display_left(t_top100_pref.head(), v_top100_pref.head())

,응답자 ID,파일명,스타일 선호 여부
0,368,W_06753_60_mods_M.jpg,스타일 선호
1,368,W_06686_70_hippie_M.jpg,스타일 선호
2,368,W_15453_70_hippie_M.jpg,스타일 비선호
3,368,W_06843_60_mods_M.jpg,스타일 선호
4,368,W_06896_10_sportivecasual_M.jpg,스타일 선호
,응답자 ID,파일명,스타일 선호 여부
0,368,W_04622_60_mods_M.jpg,스타일 선호
1,368,W_04678_50_ivy_M.jpg,스타일 선호
2,368,W_15791_70_hippie_M.jpg,스타일 비선호
3,368,W_16034_80_bold_M.jpg,스타일 비선호


# train data로부터 feature vector 추출

In [1]:
# 필요 패키지 import 
import os
import random
import numpy as np
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
import torchvision.transforms as transforms


# 모델 재현을 위한 랜덤시드 고정
def set_random_seed(seed_value=42):
    # Python의 기본 난수 시드 설정
    random.seed(seed_value)
    # NumPy 난수 시드 설정
    np.random.seed(seed_value)
    # PyTorch 난수 시드 설정 (CPU)
    torch.manual_seed(seed_value)
    # PyTorch 난수 시드 설정 (GPU)
    torch.cuda.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)
    # CuDNN 설정
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_random_seed()

In [2]:
# Mission 1-2에서 사용한 ResNet18 모델 구조 가져오기
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != self.expansion * out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, self.expansion * out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * out_channels)
            )

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = self.relu(out)
        return out

class ResNet18(nn.Module):
    def __init__(self, num_classes=31):
        super(ResNet18, self).__init__()
        self.in_channels = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(BasicBlock, 64, 2)
        self.layer2 = self._make_layer(BasicBlock, 128, 2, stride=2)
        self.layer3 = self._make_layer(BasicBlock, 256, 2, stride=2)
        self.layer4 = self._make_layer(BasicBlock, 512, 2, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * BasicBlock.expansion, num_classes)

        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='relu')
                nn.init.constant_(m.bias, 0)

    def _make_layer(self, block, out_channels, blocks, stride=1):
        layers = []
        layers.append(block(self.in_channels, out_channels, stride))
        self.in_channels = out_channels * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

In [3]:
# 학습된 가중치로 모델 불러오기
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

model = ResNet18(num_classes=31)    # 모델 생성
model.load_state_dict(torch.load('./best_model.pth', weights_only=True))    # 가중치 로드
model = model.to(device)
model.eval()        # 학습을 하는 것이 아니므로 eval 모드로 설정

cuda


ResNet18(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  

In [4]:
# 특징 추출을 위한 Class 선언
class FeatureExtractor(nn.Module):
    def __init__(self, original_model):
        super(FeatureExtractor, self).__init__()
        self.features = nn.Sequential(*list(original_model.children())[:-1])    # 마지막 fc layer 제외
    
    def forward(self, x):
        x = self.features(x)
        return torch.flatten(x, 1)

feature_extractor = FeatureExtractor(model)
feature_extractor = feature_extractor.to(device)

In [5]:
# 이미지 전처리 함수(모델 학습 시 적용한 전처리와 동일하게 적용)
def _preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    image = Image.open(image_path).convert('RGB')
    return transform(image).unsqueeze(0)

# feature vector 추출 함수
def _extract_features(image_path):
    image = _preprocess_image(image_path).to(device)
    with torch.no_grad():
        features = feature_extractor(image)
    return features.cpu().numpy().flatten()

# 여러 이미지로부터 feature vector 추출
def extract_features_from_images(dir_path, image_list):
    features = {}
    for img in image_list:
        path = os.path.join(dir_path, img)
        feature = _extract_features(path)
        features[img] = np.array(feature)
    return features

In [10]:
# 모든 train image에 대한 feature vector 추출(시간이 꽤 걸릴 수 있음)
t_img_dir = '../dataset/training_image'
t_img_list = os.listdir(t_img_dir)

extracted_features = extract_features_from_images(t_img_dir, t_img_list)

In [11]:
# feature의 shape 확인(fc layer 이전은 512개의 feature)
print(f"이미지 수: {len(extracted_features)}, feature shape: {extracted_features[t_img_list[0]].shape}")

이미지 수: 4070, feature shape: (512,)


In [12]:
# 추출된 특징을 데이터프레임으로 변환 후 저장
t_feature_vectors = pd.DataFrame(extracted_features).T
#t_feature_vectors.to_csv('train_feature_vectors.csv')
t_feature_vectors

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
T_00253_60_popart_W.jpg,1.057588,0.499090,0.964764,1.991788,0.861173,0.732690,0.020497,1.579639,2.082400,1.172687,...,0.430914,2.368689,0.886840,0.572086,0.033725,0.119491,1.169548,0.377159,0.020128,0.139832
T_00456_10_sportivecasual_M.jpg,0.705563,0.464909,0.736676,0.601670,1.329381,1.628532,0.797708,0.149928,0.418592,0.828327,...,1.245656,0.928324,1.179201,1.576541,0.382095,0.652732,0.740255,0.341360,0.011737,0.868388
T_00588_10_sportivecasual_M.jpg,0.280607,1.900605,0.076298,0.313563,0.938350,3.079298,0.057795,0.039337,0.623045,1.935424,...,2.356834,0.991338,1.462100,0.746372,0.232039,0.448638,0.771779,1.735500,0.952510,1.359725
T_00770_60_minimal_W.jpg,0.416201,1.589971,0.462377,1.117409,0.230992,0.833754,0.477335,1.091138,0.284210,1.001748,...,0.853099,0.660150,0.992873,0.510357,0.281411,0.485778,0.390882,0.790282,1.029764,0.378048
T_00893_90_hiphop_W.jpg,1.480315,1.115098,1.787921,0.440522,1.454214,0.438948,0.314583,0.311655,1.567511,1.217266,...,2.076454,1.226675,1.402020,1.062232,0.992168,0.092353,1.047830,0.154575,0.116192,0.618304
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
W_71923_60_mods_M.jpg,0.185352,0.538911,0.294497,0.886722,0.758606,0.501546,0.323596,1.483522,0.892440,1.113719,...,1.255762,0.779979,1.605300,1.090727,0.511891,0.438244,0.434495,0.188024,1.116629,0.613483
W_71933_60_mods_M.jpg,0.308852,0.314811,0.272143,0.815783,0.539355,1.296929,0.430950,1.022277,0.521891,0.679997,...,0.785857,0.480779,1.326938,0.378008,0.822803,0.701973,0.348771,0.191141,0.979934,1.377376
W_71934_60_mods_M.jpg,0.161009,0.474238,0.739945,0.385359,0.705802,0.236488,0.879010,2.100184,0.587470,0.627063,...,1.466490,0.781835,2.687330,0.803959,0.546261,0.539433,0.416490,0.315864,1.944892,0.463204
W_71935_60_mods_M.jpg,0.368237,0.493736,0.818316,0.622802,0.770126,0.752600,0.617757,1.709467,0.818570,0.821341,...,0.801818,0.785893,2.120833,0.632633,0.727804,0.279642,0.538590,0.084149,1.645151,0.717943


# 이미지 유사도를 활용한 스타일 선호 예측

In [13]:
# top100 선호도 데이터 로드
t_top100_pref = pd.read_csv('top100_train_preference.csv')
v_top100_pref = pd.read_csv('top100_val_preference.csv')

t_top100_pref['스타일 선호 여부'] = t_top100_pref['스타일 선호 여부'].apply(lambda x: 1 if x == '스타일 선호' else 0)
v_top100_pref['스타일 선호 여부'] = v_top100_pref['스타일 선호 여부'].apply(lambda x: 1 if x == '스타일 선호' else 0)

In [15]:
# feature vector 파일 각종 전처리
t_feature = pd.read_csv('train_feature_vectors.csv', index_col=0)
t_feature['feature_vector'] = t_feature.values.tolist()     # 512차원의 feature vector를 하나의 list로 변환
t_feature_reset = t_feature.reset_index().rename(columns={'index': '파일명'})
t_feature_simplified = t_feature_reset[['파일명', 'feature_vector']]    # 파일명과 feature vector만 남기기
t_feature_simplified

,파일명,feature_vector
0,T_00253_60_popart_W.jpg,"[1.0575885, 0.49908996, 0.96476394, 1.9917883,..."
1,T_00456_10_sportivecasual_M.jpg,"[0.7055633, 0.4649085, 0.7366763, 0.60167015, ..."
2,T_00588_10_sportivecasual_M.jpg,"[0.28060737, 1.9006052, 0.07629789, 0.31356296..."
3,T_00770_60_minimal_W.jpg,"[0.41620094, 1.589971, 0.46237656, 1.1174088, ..."
4,T_00893_90_hiphop_W.jpg,"[1.4803153, 1.1150981, 1.7879208, 0.4405219, 1..."
...,...,...
4065,W_71923_60_mods_M.jpg,"[0.1853519, 0.5389108, 0.2944971, 0.8867223, 0..."
4066,W_71933_60_mods_M.jpg,"[0.3088518, 0.31481087, 0.2721426, 0.8157828, ..."
4067,W_71934_60_mods_M.jpg,"[0.16100864, 0.47423834, 0.7399452, 0.38535923..."
4068,W_71935_60_mods_M.jpg,"[0.3682367, 0.49373567, 0.8183162, 0.6228021, ..."


In [16]:
# top100 선호도 데이터에 feature vector를 병합(left merge)
t_merged_df = pd.merge(t_top100_pref, t_feature_simplified, on='파일명', how='left')
t_merged_df

,응답자 ID,파일명,스타일 선호 여부,feature_vector
0,368,W_06753_60_mods_M.jpg,1,"[0.70436937, 0.5913673, 1.950396, 0.84063923, ..."
1,368,W_06686_70_hippie_M.jpg,1,"[0.022540793, 0.39496934, 0.7694336, 0.0089152..."
2,368,W_15453_70_hippie_M.jpg,0,"[0.10546503, 0.09943376, 0.7443662, 0.8763011,..."
3,368,W_06843_60_mods_M.jpg,1,"[0.36511362, 0.88988304, 0.31558028, 0.4499632..."
4,368,W_06896_10_sportivecasual_M.jpg,1,"[0.49174654, 1.4860588, 1.1976274, 0.31256232,..."
...,...,...,...,...
4449,67975,W_07095_00_metrosexual_M.jpg,1,"[0.7742443, 1.3371912, 0.8389899, 0.722591, 1...."
4450,67975,T_21986_70_hippie_M.jpg,0,"[0.69457424, 0.4065542, 1.262427, 0.61637765, ..."
4451,67975,T_21987_70_hippie_M.jpg,1,"[0.22798271, 0.7277869, 1.1448684, 0.23363097,..."
4452,67975,T_17800_19_normcore_M.jpg,1,"[0.9351864, 1.3963087, 2.6302078, 1.1111004, 1..."


In [39]:
# 유사도 계산(코사인 유사도)
from sklearn.metrics.pairwise import cosine_similarity

pred_list = []

for i in range(len(v_top100_pref)):
    userID = v_top100_pref.loc[i, '응답자 ID']
    img_name = v_top100_pref.loc[i, '파일명']

    # v_top100_pref의 img에 대한 feature vector 추출
    val_img_path = "../dataset/validation_image/"
    feature_vector = _extract_features(val_img_path + img_name)

    # train image 중 해당 유저가 이미 평가한 image들만 추출
    user_evaluated_items = t_merged_df.loc[t_merged_df['응답자 ID']==userID, :]

    # 해당 유저가 평가한 image들과 v_top100_pref의 img에서 추출한 feature vector와 유사도 계산(코사인 유사도)
    similarity = cosine_similarity([feature_vector], user_evaluated_items['feature_vector'].tolist())

    # 유사도 벡터를 DataFrame으로 변환하여 직관적으로 보기 쉽게 변환
    t_item_similarity_df = pd.DataFrame(similarity, index=[img_name], columns=user_evaluated_items['파일명'])

    # 유사도가 가장 높은 이미지 추출 (!상위 5개에서 voting을 하는 등 추천 방식은 변환 가능할 것)
    top_similar_item = t_item_similarity_df.T.sort_values(by=img_name, ascending=False).head(1).index[0]

    # top_similar_item의 선호 여부 확인 -> 해당 값을 추천 결과로 사용
    top_similar_item_pref = user_evaluated_items[user_evaluated_items['파일명'] == top_similar_item]['스타일 선호 여부'].values[0]

    # 예측값 저장
    pred_list.append(top_similar_item_pref)

In [40]:
# v_top100_pref에 대한 예측값을 열에 추가
v_top100_pref['선호도 예측'] = pred_list
v_top100_pref

,응답자 ID,파일명,스타일 선호 여부,선호도 예측
0,368,W_04622_60_mods_M.jpg,1,1
1,368,W_04678_50_ivy_M.jpg,1,1
2,368,W_15791_70_hippie_M.jpg,0,0
3,368,W_16034_80_bold_M.jpg,0,0
4,368,W_06551_60_mods_M.jpg,0,0
...,...,...,...,...
1097,67975,W_07074_00_metrosexual_M.jpg,1,1
1098,67975,W_52578_50_ivy_M.jpg,1,1
1099,67975,W_17742_80_bold_M.jpg,0,0
1100,67975,W_26965_90_hiphop_M.jpg,0,0


In [116]:
# 성능지표 평가 (Accuracy / Precision / Recall)
from sklearn.metrics import accuracy_score, precision_score, recall_score

accuracy = accuracy_score(v_top100_pref['스타일 선호 여부'], v_top100_pref['선호도 예측'])
precision = precision_score(v_top100_pref['스타일 선호 여부'], v_top100_pref['선호도 예측'])
recall = recall_score(v_top100_pref['스타일 선호 여부'], v_top100_pref['선호도 예측'])

print(f"Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")

Accuracy: 0.8367, Precision: 0.7991, Recall: 0.7919
